# Session 3 - Maîtriser l'Attention

**Module** : Réseaux de Neurones Approfondissement  
**Durée** : 2h  
**Objectif** : Implémenter l'attention de A à Z et découvrir le Multi-Head

---

## Objectifs pédagogiques

À la fin de cette session, vous serez capable de :
1. Implémenter la fonction `scaled_dot_product_attention()` complète
2. Créer une classe `SelfAttention` réutilisable en PyTorch
3. Visualiser et interpréter l'attention d'un vrai modèle (CamemBERT)
4. Comprendre pourquoi on utilise **plusieurs têtes** d'attention

---

## Rappel : Où en sommes-nous ?

Dans la session précédente, vous avez :
- Compris le **Positional Encoding** (encoder l'ordre des mots)
- Vu le lien entre **similarité** et **produit scalaire**
- Découvert les concepts de **Query, Key, Value**
   - Q : Ce que je cherche
   - K : Les étiquettes
   - V : Le contenu
- Calculé les scores d'attention étape par étape

Aujourd'hui, on passe à l'**implémentation complète** !

## 0. Installation et imports

In [ ]:
# Installation des dépendances (Google Colab)
!pip install torch matplotlib numpy transformers bertviz -q

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import math

torch.manual_seed(42)
print(f"PyTorch version: {torch.__version__}")

---

## 1. Rappel : La formule de l'attention

$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

| Étape | Opération | Rôle |
|-------|-----------|------|
| 1 | $QK^T$ | Calculer les scores de similarité |
| 2 | $\div \sqrt{d_k}$ | Stabiliser les gradients |
| 3 | softmax | Transformer en probabilités |
| 4 | $\times V$ | Moyenne pondérée des values |

Dans la session précédente, vous avez fait ces étapes **séparément**. Maintenant, on les **combine** dans une fonction.

---

## 2. Exercice 1 : Fonction d'attention complète

Regroupez les 4 étapes dans une seule fonction réutilisable.

In [ ]:
def scaled_dot_product_attention(Q, K, V):
    """
    Calcule le Scaled Dot-Product Attention.
    
    Args:
        Q: Queries, shape (..., seq_len, d_k)
        K: Keys, shape (..., seq_len, d_k)
        V: Values, shape (..., seq_len, d_v)
    
    Returns:
        output: Résultat de l'attention, même shape que V
        attention_weights: Poids d'attention, shape (..., seq_len, seq_len)
    """
    # CORRECTION 1: Récupérer d_k (dernière dimension de K)
    d_k = K.shape[-1]
    
    # CORRECTION 2: Calculer les scores QK^T
    scores = Q @ K.transpose(-2, -1)
    
    # CORRECTION 3: Appliquer le scaling (diviser par sqrt(d_k))
    scaled_scores = scores / math.sqrt(d_k)
    
    # CORRECTION 4: Appliquer softmax sur la dernière dimension
    attention_weights = F.softmax(scaled_scores, dim=-1)
    
    # CORRECTION 5: Calculer la sortie (weights @ V)
    output = attention_weights @ V
    
    return output, attention_weights

In [ ]:
# Test de votre fonction
Q_test = torch.randn(4, 8)  # 4 tokens, dimension 8
K_test = torch.randn(4, 8)
V_test = torch.randn(4, 8)

output, weights = scaled_dot_product_attention(Q_test, K_test, V_test)

print(f"Output shape: {output.shape}")    # Attendu: (4, 8)
print(f"Weights shape: {weights.shape}")  # Attendu: (4, 4)
print(f"Somme des poids par ligne: {weights.sum(dim=-1)}")  # Attendu: [1, 1, 1, 1]

### Vérification : Effet du scaling

Pourquoi diviser par $\sqrt{d_k}$ ? Voyons l'effet.

In [ ]:
# Comparaison avec et sans scaling
d_k_grand = 512  # Dimension typique dans un Transformer

Q_grand = torch.randn(10, d_k_grand)
K_grand = torch.randn(10, d_k_grand)

# Sans scaling
scores_sans = Q_grand @ K_grand.T
attention_sans = F.softmax(scores_sans, dim=-1)

# Avec scaling
scores_avec = (Q_grand @ K_grand.T) / math.sqrt(d_k_grand)
attention_avec = F.softmax(scores_avec, dim=-1)

print("=== SANS SCALING ===")
print(f"Scores - min: {scores_sans.min():.1f}, max: {scores_sans.max():.1f}")
print(f"Attention max par ligne: {attention_sans.max(dim=-1).values[:5]}")

print("\n=== AVEC SCALING ===")
print(f"Scores - min: {scores_avec.min():.1f}, max: {scores_avec.max():.1f}")
print(f"Attention max par ligne: {attention_avec.max(dim=-1).values[:5]}")

print("\nAvec scaling, l'attention est mieux répartie (pas de valeur proche de 1)")

---

## 3. Exercice 2 : Classe SelfAttention

### D'où viennent Q, K, V ?

Jusqu'ici, on a utilisé des tenseurs aléatoires. En pratique, **Q, K, V sont calculés à partir des embeddings** via des matrices apprenables.

```
x (embeddings) ──┬──► W_q ──► Q   (ce que je cherche)
                 ├──► W_k ──► K   (comment je me présente)
                 └──► W_v ──► V   (l'info que je transmets)
```

### Pourquoi 3 matrices différentes ?

Si on utilisait directement `x @ x.T`, on calculerait les **similarités sémantiques** entre mots. "Pikachu" serait attentif à "Raichu", "électrique"...

Avec des projections différentes (W_q, W_k, W_v), le modèle peut capturer d'autres types de relations : **syntaxiques** (sujet-verbe), **contextuelles**, etc.

Les matrices sont **apprises** pendant l'entraînement : elles s'ajustent pour que la formule `softmax(Q @ K.T) @ V` produise des résultats utiles pour la tâche.

> **Pour approfondir** : voir la note *"Comprendre le Mécanisme d'Attention"* qui détaille le rôle de Q, K, V et comment les matrices apprennent.

In [ ]:
class SelfAttention(nn.Module):
    """
    Module de Self-Attention.
    
    Projette l'input x vers Q, K, V puis applique l'attention.
    """
    
    def __init__(self, embed_dim):
        """
        Args:
            embed_dim: Dimension des embeddings d'entrée
        """
        super().__init__()
        self.embed_dim = embed_dim
        
        # CORRECTION 1: Créer 3 couches linéaires pour projeter vers Q, K, V
        self.W_q = nn.Linear(embed_dim, embed_dim)
        self.W_k = nn.Linear(embed_dim, embed_dim)
        self.W_v = nn.Linear(embed_dim, embed_dim)
    
    def forward(self, x):
        """
        Args:
            x: Embeddings, shape (batch, seq_len, embed_dim)
        
        Returns:
            output: Résultat de l'attention
            attention_weights: Poids d'attention
        """
        # CORRECTION 2: Projeter x vers Q, K, V
        Q = self.W_q(x)
        K = self.W_k(x)
        V = self.W_v(x)
        
        # CORRECTION 3: Appliquer la fonction scaled_dot_product_attention
        output, attention_weights = scaled_dot_product_attention(Q, K, V)
        
        return output, attention_weights

In [ ]:
# Test du module
embed_dim = 32
batch_size = 2
seq_len = 5

attention_layer = SelfAttention(embed_dim)
x = torch.randn(batch_size, seq_len, embed_dim)

output, weights = attention_layer(x)

print(f"Input shape: {x.shape}")
print(f"Output shape: {output.shape}")   # Attendu: (2, 5, 32)
print(f"Weights shape: {weights.shape}") # Attendu: (2, 5, 5)

# Compter les paramètres
n_params = sum(p.numel() for p in attention_layer.parameters())
print(f"\nNombre de paramètres: {n_params:,}")

---

## 4. Visualisation sur CamemBERT

Maintenant qu'on a compris et implémenté l'attention, regardons ce que ça donne sur un modèle **réellement entraîné**.

On utilise **CamemBERT**, le modèle français qu'on a découvert au TP précédent, avec notre corpus Pokémon !

### Tokens spéciaux CamemBERT

| Token | Rôle |
|-------|------|
| **\<s\>** | Début de phrase (équivalent [CLS]) |
| **\</s\>** | Fin de phrase (équivalent [SEP]) |

### Choix de la couche et de la tête

CamemBERT a **12 couches** et **12 têtes par couche** = 144 matrices d'attention différentes !

Toutes ne sont pas intéressantes à visualiser. On va explorer différentes têtes pour voir lesquelles capturent la **coréférence** (le lien entre un pronom et son antécédent).

In [ ]:
from transformers import CamembertModel, CamembertTokenizer

# Charger CamemBERT
print("Chargement de CamemBERT...")
tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
model = CamembertModel.from_pretrained("camembert-base", output_attentions=True)
model.eval()

print(f"Modèle chargé !")
print(f"  - Couches : {model.config.num_hidden_layers}")
print(f"  - Têtes par couche : {model.config.num_attention_heads}")

In [ ]:
# Phrase Pokémon avec coréférence
phrase = "Pikachu a utilisé Tonnerre sur Dracaufeu car il était très efficace"

# Tokeniser
inputs = tokenizer(phrase, return_tensors="pt")
tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

# Forward pass
with torch.no_grad():
    outputs = model(**inputs)

attentions = outputs.attentions  # tuple de 12 tenseurs (1 par couche)

print(f"Phrase: {phrase}")
print(f"\nTokens ({len(tokens)}): {tokens}")
print(f"\nStructure des attentions:")
print(f"  - Nombre de couches: {len(attentions)}")
print(f"  - Shape par couche: {attentions[0].shape}  (batch, heads, seq, seq)")

### 4.1 Visualisation avec Matplotlib

Commençons par une visualisation classique pour comprendre la structure.

In [ ]:
def plot_attention(attention_matrix, tokens, title="Attention"):
    """Affiche une matrice d'attention avec matplotlib."""
    plt.figure(figsize=(10, 8))
    plt.imshow(attention_matrix, cmap='Blues')
    plt.xticks(range(len(tokens)), tokens, rotation=45, ha='right')
    plt.yticks(range(len(tokens)), tokens)
    plt.xlabel("Tokens regardés (Keys)")
    plt.ylabel("Tokens qui regardent (Queries)")
    plt.title(title)
    plt.colorbar(label="Poids d'attention")
    
    # Ajouter les valeurs
    for i in range(len(tokens)):
        for j in range(len(tokens)):
            val = attention_matrix[i, j]
            color = 'white' if val > 0.3 else 'black'
            plt.text(j, i, f'{val:.2f}', ha='center', va='center',
                    color=color, fontsize=7)
    plt.tight_layout()
    plt.show()

In [ ]:
# Visualiser une tête - Couche 8, Tête 10 (souvent intéressante pour la coréférence)
layer = 7  # 0-indexed
head = 9   # 0-indexed

attention_matrix = attentions[layer][0, head].numpy()
plot_attention(attention_matrix, tokens, f"CamemBERT - Couche {layer+1}, Tête {head+1}")

In [ ]:
# Que regarde le pronom "il" ?
# Trouver l'index de "il" dans les tokens
il_index = None
for i, t in enumerate(tokens):
    if "il" in t.lower():
        il_index = i
        break

if il_index:
    print(f"Attention de '{tokens[il_index]}' (Couche {layer+1}, Tête {head+1}) :")
    print("-" * 50)
    
    for i, (token, weight) in enumerate(zip(tokens, attention_matrix[il_index])):
        bar = "*" * int(weight * 30)
        highlight = ""
        if "Pikachu" in token or "pikachu" in token.lower():
            highlight = " <-- Pikachu ?"
        elif "Tonnerre" in token or "tonnerre" in token.lower():
            highlight = " <-- Tonnerre ?"
        print(f"  {token:15} {weight:.3f} {bar}{highlight}")
else:
    print("Token 'il' non trouvé")

### 4.2 Visualisation interactive avec BertViz

**BertViz** est un outil spécialisé pour visualiser l'attention des Transformers. Il offre des vues interactives beaucoup plus riches que matplotlib.

Trois modes de visualisation :
- **head_view** : voir les connexions d'attention pour une couche
- **model_view** : vue globale de toutes les couches et têtes
- **neuron_view** : voir comment Q et K contribuent aux scores

In [ ]:
from bertviz import head_view, model_view

# Préparer les attentions pour BertViz
# BertViz attend une liste/tuple de tenseurs, ce qu'on a déjà !

print("head_view : visualisation interactive d'une couche")
print("Cliquez sur une tête pour voir ses connexions d'attention")
print("="*50)

In [ ]:
# Head View - Vue par tête (interactif)
# Affiche toutes les têtes d'une couche avec les liens d'attention
head_view(attentions, tokens)

In [ ]:
# Model View - Vue globale du modèle
# Permet de naviguer dans toutes les couches et têtes
print("model_view : vue globale de toutes les couches")
print("Sélectionnez une couche et une tête pour explorer")
print("="*50)

model_view(attentions, tokens)

### 4.3 Explorer d'autres phrases Pokémon

In [ ]:
def analyser_attention_pokemon(phrase, mot_cible=None):
    """
    Analyse l'attention de CamemBERT sur une phrase Pokémon.
    
    Args:
        phrase: La phrase à analyser
        mot_cible: Le mot dont on veut voir l'attention (optionnel)
    """
    inputs = tokenizer(phrase, return_tensors="pt")
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    attentions = outputs.attentions
    
    print(f"Phrase: {phrase}")
    print(f"Tokens: {tokens}\n")
    
    # Afficher avec BertViz
    return head_view(attentions, tokens)

In [ ]:
# Essayez différentes phrases !
phrases_pokemon = [
    "Le dresseur a rappelé Salamèche car il était trop faible",
    "Évoli a évolué en Aquali parce qu'il a touché une Pierre Eau",
    "Sacha a attrapé un Rondoudou et il était très content",
]

# Analysez la première phrase
analyser_attention_pokemon(phrases_pokemon[0])

---

## 5. Pourquoi Multi-Head Attention ?

### Le problème avec une seule tête

Une seule tête d'attention calcule **une** représentation des relations entre mots.

Mais dans une phrase, il y a **plusieurs types de relations** :
- Relations **syntaxiques** (sujet-verbe)
- Relations **sémantiques** (sens, coréférence)
- Relations de **proximité** (mots voisins)
- etc.

### La solution : plusieurs têtes

Chaque tête peut apprendre à détecter un type de relation différent !

**Attention** : En pratique, une tête ne correspond pas forcément à un type de relation précis. Une relation peut être portée par une combinaison de têtes, et une tête peut capturer plusieurs types de patterns. C'est ce que le modèle apprend pendant l'entraînement.

**Analogie** : C'est comme avoir plusieurs experts qui analysent une phrase sous différents angles, puis combinent leurs analyses.

### Exemple : comparons deux têtes de CamemBERT

In [ ]:
# Reprenons notre phrase Pokémon
phrase = "Pikachu a utilisé Tonnerre sur Dracaufeu car il était très efficace"
inputs = tokenizer(phrase, return_tensors="pt")
tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

with torch.no_grad():
    outputs = model(**inputs)
attentions = outputs.attentions

# Comparaison de 2 têtes
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Tête des couches profondes (relations sémantiques)
head_semantic = attentions[9][0, 5].numpy()  # Couche 10, Tête 6

# Tête des premières couches (attention locale)
head_local = attentions[1][0, 0].numpy()  # Couche 2, Tête 1

for idx, (attn, title) in enumerate([
    (head_semantic, "Couche 10, Tête 6\n(couches profondes)"),
    (head_local, "Couche 2, Tête 1\n(attention plus locale)")
]):
    ax = axes[idx]
    im = ax.imshow(attn, cmap='Blues')
    ax.set_xticks(range(len(tokens)))
    ax.set_xticklabels(tokens, rotation=45, ha='right', fontsize=9)
    ax.set_yticks(range(len(tokens)))
    ax.set_yticklabels(tokens, fontsize=9)
    ax.set_title(title, fontsize=12)
    plt.colorbar(im, ax=ax)

plt.suptitle("Deux têtes de CamemBERT capturent des patterns différents", fontsize=13, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Trouver l'index de "il"
il_index = None
for i, t in enumerate(tokens):
    if "il" in t.lower():
        il_index = i
        break

if il_index:
    print(f"Que regarde '{tokens[il_index]}' selon chaque tête ?")
    print("=" * 50)
    
    print(f"Tête sémantique (C10-T6) : ", end="")
    for i, w in enumerate(head_semantic[il_index]):
        if w > 0.08:
            print(f"{tokens[i]}({w:.2f}) ", end="")
    print()
    
    print(f"Tête locale (C2-T1) :      ", end="")
    for i, w in enumerate(head_local[il_index]):
        if w > 0.08:
            print(f"{tokens[i]}({w:.2f}) ", end="")
    print()

---

## 6. Architecture Multi-Head Attention

### Schéma

```
Input (seq_len, embed_dim)
        |
   +----+----+--------+--------+
   |         |        |        |
   v         v        v        v
 Head 1   Head 2   Head 3   Head 4
   |         |        |        |
   +----+----+--------+--------+
        |
    Concat
        |
   Linear (W_o)
        |
Output (seq_len, embed_dim)
```

### Dimensions

| Paramètre | Exemple | Description |
|-----------|---------|-------------|
| embed_dim | 512 | Dimension des embeddings |
| num_heads | 8 | Nombre de têtes |
| d_k | 64 | Dimension par tête = embed_dim / num_heads |

Chaque tête travaille avec une **dimension réduite** ($d_k$), puis on **concatène** les résultats.

### Pourquoi W_o après le concat ?

Après concaténation, on a un vecteur de dimension `embed_dim` (4 têtes x 64 = 512).

La matrice **W_o** (output projection) permet de :
1. **Mélanger** les informations des différentes têtes
2. **Apprendre** comment combiner leurs "points de vue"

Sans W_o, les têtes resteraient indépendantes. Avec W_o, le modèle peut apprendre des combinaisons utiles.

## 7. Exercice 3 : split_heads (Multi-Head)

La première étape du Multi-Head est de **séparer les têtes**.

On transforme `(batch, seq_len, embed_dim)` en `(batch, num_heads, seq_len, d_k)`

In [ ]:
# Configuration
batch_size = 2
seq_len = 6
embed_dim = 32
num_heads = 4
d_k = embed_dim // num_heads  # 32 / 4 = 8

print(f"embed_dim: {embed_dim}")
print(f"num_heads: {num_heads}")
print(f"d_k (dim par tête): {d_k}")

# Input
x = torch.randn(batch_size, seq_len, embed_dim)
print(f"\nInput shape: {x.shape}")

In [ ]:
def split_heads(x, num_heads):
    """
    Sépare les têtes d'attention.
    
    Reshape: (batch, seq_len, embed_dim) -> (batch, num_heads, seq_len, d_k)
    
    Args:
        x: Tensor de shape (batch, seq_len, embed_dim)
        num_heads: Nombre de têtes
    
    Returns:
        Tensor de shape (batch, num_heads, seq_len, d_k)
    """
    batch_size, seq_len, embed_dim = x.shape
    d_k = embed_dim // num_heads
    
    # CORRECTION:
    # Étape 1: Séparer embed_dim en (num_heads, d_k)
    x = x.view(batch_size, seq_len, num_heads, d_k)
    # Étape 2: Réorganiser pour avoir num_heads en position 1
    x = x.transpose(1, 2)
    
    return x

In [ ]:
# Test
x_test = torch.randn(2, 6, 32)  # batch=2, seq=6, embed=32
x_heads = split_heads(x_test, num_heads=4)

print(f"Avant split: {x_test.shape}")   # (2, 6, 32)
print(f"Après split: {x_heads.shape}")  # Attendu: (2, 4, 6, 8)

if x_heads is not None and x_heads.shape == (2, 4, 6, 8):
    print("\nCorrect !")
else:
    print("\nVérifiez votre implémentation")

---

## 8. Récapitulatif

### Ce que nous avons appris aujourd'hui

| Concept | Ce qu'on a fait |
|---------|----------------|
| **Exercice 1** | Fonction `scaled_dot_product_attention()` complète |
| **Exercice 2** | Classe `SelfAttention` avec projections W_q, W_k, W_v |
| **Visualisation** | Explorer l'attention de CamemBERT avec BertViz |
| **Multi-Head** | Comprendre pourquoi plusieurs têtes |
| **Exercice 3** | Fonction `split_heads()` pour séparer les têtes |

### Questions de compréhension

Avant de passer au TP suivant, vérifiez que vous pouvez répondre à ces questions :

1. **Scaling** : Que se passe-t-il si on ne divise pas par sqrt(d_k) ? Pourquoi ?

2. **Softmax** : Pourquoi la somme des poids d'attention fait toujours 1 ?

3. **Paramètres** : Combien de paramètres a une couche `SelfAttention` avec `embed_dim=512` ? (indice : 3 matrices W_q, W_k, W_v)

4. **Multi-Head** : Si on a `embed_dim=512` et `num_heads=8`, quelle est la dimension d_k de chaque tête ?

5. **Q != K** : Pourquoi utilise-t-on des matrices différentes pour Q et K au lieu de faire simplement `x @ x.T` ?

<details>
<summary>Réponses</summary>

1. Les scores deviennent très grands -> softmax donne des poids proches de 0 ou 1 -> gradients instables

2. C'est la propriété du softmax : il transforme des scores en probabilités qui somment à 1

3. 3 x (512 x 512 + 512) = 3 x 262 656 = **787 968** paramètres (poids + biais)

4. d_k = 512 / 8 = **64**

5. Pour permettre au modèle de capturer d'autres relations que la similarité sémantique (relations syntaxiques, contextuelles, etc.)
</details>

### Prochaine session

On va :
1. Terminer l'implémentation Multi-Head (`concat_heads`, classe complète)
2. Ajouter le **Feed-Forward Network**
3. Comprendre le **masque causal** (GPT vs BERT)
4. Assembler un **bloc Transformer** complet !

---

## 9. Pour aller plus loin (optionnel)

### 9.1 Explorer toutes les têtes d'une couche

In [ ]:
# Visualiser 4 têtes différentes de la même couche
fig, axes = plt.subplots(2, 2, figsize=(14, 12))

heads_to_show = [0, 3, 7, 11]
layer = 8  # Couche 9

for idx, head in enumerate(heads_to_show):
    ax = axes[idx // 2, idx % 2]
    w = attentions[layer][0, head].numpy()
    
    im = ax.imshow(w, cmap='Blues')
    ax.set_xticks(range(len(tokens)))
    ax.set_xticklabels(tokens, rotation=45, ha='right', fontsize=8)
    ax.set_yticks(range(len(tokens)))
    ax.set_yticklabels(tokens, fontsize=8)
    ax.set_title(f"Tête {head + 1}", fontsize=11)
    plt.colorbar(im, ax=ax)

plt.suptitle(f"Différentes têtes de la couche {layer+1} - CamemBERT", fontsize=13)
plt.tight_layout()
plt.show()

In [ ]:
# Analyse : que regarde "il" selon chaque tête ?
if il_index:
    print(f"Que regarde '{tokens[il_index]}' selon chaque tête de la couche {layer+1} ?")
    print("=" * 60)
    
    for head in heads_to_show:
        weights = attentions[layer][0, head, il_index].numpy()
        top_idx = weights.argsort()[-3:][::-1]  # Top 3
        print(f"Tête {head+1:2d}: ", end="")
        for i in top_idx:
            print(f"{tokens[i]} ({weights[i]:.2f})  ", end="")
        print()

### 9.2 Comparer l'attention sur différentes phrases

Testez avec vos propres phrases Pokémon pour voir comment l'attention change !

In [ ]:
# Votre phrase ici !
ma_phrase = "Mewtwo est le Pokémon le plus puissant car il a été créé génétiquement"

analyser_attention_pokemon(ma_phrase)